In [24]:
import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sg
import sounddevice as sd
import wavio
import scipy.signal as sig
import time, math, sys
import scipy.fftpack as fft
import pyaudio
import wave
import commpy as cp
import io
import wave

# import numpy as np
# import scipy.io.wavfile
# import soundfile as sf
from scipy.io.wavfile import write

In [8]:
class Filter(object):
    def __init__(self, b, a = [1], init=[]):
        self.b = b
        self.a = a
        self.clear(init)
    def __call__(self, x):
        y, self.state = sig.lfilter(self.b, self.a, x, zi=self.state)
        return y
    def clear(self, init=[]):
        self.state = sig.lfiltic(self.b, self.a, [], init)

In [3]:
def access_bit(data, num):
    base = int(num // 8)
    shift = int(num % 8)
    return (data[base] >> shift) & 0x1

In [25]:
def convert_bytearray_to_wav_ndarray(input_bytearray: bytes, sampling_rate=44100):
    bytes_wav = bytes()
    byte_io = io.BytesIO(bytes_wav)
    write(byte_io, sampling_rate, np.frombuffer(input_bytearray, dtype=np.int16))
    output_wav = byte_io.read()
    output, samplerate = sf.read(io.BytesIO(output_wav))
    return output

In [19]:
def gen_ldpc_H(n=320, m=160):
    cmax = 10
    rmax = 20
    rng = np.random.default_rng()
    H = np.zeros((m, n))
    top_layer = np.zeros((int(m/cmax), n))
    for i in range(top_layer.shape[0]):
        for j in range(rmax):
            top_layer[i][i*rmax + j] = 1
    H[:top_layer.shape[0]] = top_layer
    
    for i in range(1,cmax):
        rand_layer = rng.permutation(top_layer, axis=1)
        H[i*top_layer.shape[0]:(i+1)*top_layer.shape[0]] = rand_layer
    return H

In [20]:
def encode(G, x):
    return x @ G %2

In [21]:
def decode(H, u, niter):
    K = H.shape[1]-H.shape[0]
    for i in range(niter):
        s = (H * u) % 2
        if np.sum(s) == 0:
            return u[:K]
        error = np.zeros(u.shape)
        error[np.argmax(s.T * H)] = 1
        u = u - error
    return u[:K]

In [30]:
def get_G_from_H(H):
    m,n = H.shape
    i=0
    j=0

    while i < m and j < n:
        k = np.argmax(H[i:, j]) +i
        temp = np.copy(H[k])
        H[k] = H[i]
        H[i] = temp
        aijn = H[i, j:]
        col = np.copy(H[:, j]) 
        col[i] = 0 
        flip = np.outer(col, aijn)
        H[:, j:] = H[:, j:] ^ flip
        i += 1
        j +=1

    perm = np.arange(n)
    return np.hstack([H[:,m:], H[:,:m]]), np.hstack([perm[m:], perm[:m]])

In [29]:
def shaffle_H_like_G(perm, H):
    idx = np.empty_like(perm)
    idx[perm] = np.arange(len(perm))
    H[:] = H[:, idx] 
    return H

In [10]:
Barker11 = [+1, +1, +1, -1, -1, -1, +1, -1, -1, +1, -1]
BarkerFilter = Filter(Barker11[::-1])
DelayFilter  = Filter([0,0,0,1])

data = b'asnfajksdfnal'
bits = [access_bit(data,i) for i in range(len(data)*8)]
bits = np.array(bits, dtype=float)
# print(bits)
#bits = np.array([1,0,1,1,1,0])
signal = 1 - 2*bits
print(len(signal))
# data = np.concatenate((Barker11, (1 - 2 * np.random.randint(2, size=256))/ np.sqrt(2)))
data = np.concatenate((Barker11, signal))

corr = BarkerFilter(data)
# corr
# в качестве трешхолда брать среднее от энергии (т.е. от квадрата data)

104


In [12]:
UP = 48
N = 4801
t, rrc = cp.filters.rrcosfilter(N, 0.25, 1e-3, 48e3)
tx = np.zeros(len(data)*UP + len(rrc)*2)
tx[:len(data)*UP:UP] = data
# tx = np.zeros(len(data)*UP)
# tx[::UP] = data
tx = tx 

UpFilter1 = Filter(rrc)
UpFilter2 = Filter(rrc/48)
# tx = DelayFilter(UpFilter1(tx) * np.exp(2j*np.pi*7e3/48e3) это отправляем на передатчик (soundfile или pyAudio)
# читаю сколько хочу, передаю в x = UpFilter2(), BarklerFilter(x) 
rx = UpFilter2(DelayFilter(UpFilter1(tx) + np.random.normal(scale=4, size=tx.shape)))
# plt.plot(20*np.log10(np.abs(np.fft.fft(rx))))
# plt.plot(np.arange(len(rx)), rx)

In [28]:
def get_sound(fileout):
    UP = 48
    BarkerInt = np.zeros(len(Barker11)*UP)
    BarkerInt[::UP] = Barker11[::-1]
    BarkerFilter = Filter(np.convolve(np.convolve(BarkerInt, rrc/48), rrc))
    barrier = 1 # нужно экспериментально как-то подобрать это значение видимо
    encoded_data = np.array([])
    while True:
        # читаем кусок данных
        data = read_data()
        rx = UpFilter2(data)
        corr = BarkerFilter(rx)

        if np.max(np.abs(corr)) < barrier:
            continue

        threshold = np.mean(rx**2)
        offset = np.argwhere(corr > threshold)[0][0]
        p = np.sign(rx[offset::UP]) # тут надо убирать барклера?
        encoded_data = np.concatenate([encoded_data, (1-p)/2]) # -1 -> 1; 1 -> 0
        if len(encoded_data) > CHUNK:
            samples = decode(encoded_data[:CHUNK])
            encoded_data = encoded_data[CHUNK:]
            # write samples to file